In [2]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import os
import glob
from nibabel.streamlines import save as save_trk
from nibabel.streamlines import Tractogram
from os import path
from dipy.tracking.local import ParticleFilteringTracking
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere, Sphere, HemiSphere
from dipy.direction import (peaks_from_model, ProbabilisticDirectionGetter)
from dipy.io import read_bvals_bvecs
from dipy.io.image import save_nifti
from dipy.io.streamline import save_trk, load_trk
from dipy.reconst.peaks import reshape_peaks_for_visualization
from dipy.reconst.dti import TensorModel, fractional_anisotropy, mean_diffusivity
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,auto_response, recursive_response, AxSymShResponse)
from dipy.segment.mask import median_otsu
from dipy.tracking.local import (LocalTracking, ThresholdTissueClassifier, ActTissueClassifier, CmcTissueClassifier)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask, move_streamlines
from dipy.viz import actor, window
from dipy.tracking.utils import target, density_map
from dipy.reconst.shm import lazy_index, real_sph_harm
from dipy.core.geometry import vec2vec_rotmat, cart2sphere
from dipy.tracking import distances
from dipy.core.ndindex import ndindex

In [19]:
os.chdir('/space/hemera/1/users/cmaffei/data_nancy/kancon03/')
wd = '/space/hemera/1/users/cmaffei/data_nancy/kancon03/'

In [5]:
# Import DWI data, bvec, bval, and binary mask
fimg =  ('/space/neptune/1/users/srf29/diffusion/kanwishercon/db/vols_dti/mgh/kancon03/dti/data.nii.gz')
img = nib.load(fimg)
data = img.get_data()
print("Data Shape: " + str(data.shape))
affine = img.affine #no need to store the affine tho. 

#Bval and bvec file information import
fbval= ('bvals')
fbvec = ("bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)
#Read the voxel size from the image header.
voxel_size = img.header.get_zooms()[:3]
print ('Voxel Size: ' + str(voxel_size))

#importing binary mask
mask=("/space/neptune/1/users/srf29/diffusion/kanwishercon/db/vols_dti/mgh/kancon03/dti/nodif_brain_mask.nii.gz")
img = nib.load(mask)
mask = img.get_data()
#print('mask.shape (%d, %d, %d)' % mask.shape)
print("Mask Shape: " + str(mask.shape))

Data Shape: (140, 140, 78, 272)
Voxel Size: (1.8, 1.8, 1.8000001)
Mask Shape: (140, 140, 78)


In [ ]:
# Import DWI data, bvec, bval, and binary mask
fimg =  ('1k/mgh_1015/data_corr.nii.gz')
img = nib.load(fimg)
data_1k = img.get_data()
print("Data Shape: " + str(data_1k.shape))
affine = img.affine #no need to store the affine tho. 

#Bval and bvec file information import
fbval= ('1k/mgh_1015/bvals')
fbvec = ("1k/mgh_1015/bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab_1k = gradient_table(bvals,bvecs)

In [ ]:
data_small=data[50:90,65:75,50:80]
data_1k_small=data_1k[50:90,65:75,50:80]
mask_small=mask[50:90,65:75,50:80]

In [33]:
trace = 0.0021
fa = 0.1
peak_thr = 0.08
convergence = 0.001
S0 = 1.
sh_order = 8
sphere = get_sphere('symmetric642')
min_sep_angle = 25
parallel = True
nbr_processes = 4
iter = 8

In [7]:
#importing binary mask
wm_mask=("t12fa_pve_2_res.nii.gz")
img = nib.load(wm_mask)
wm_mask = img.get_data()
wm_mask=wm_mask>0.8
nib.save(nib.Nifti1Image(wm_mask.astype('float32'), img.affine, img.header), os.path.join(wd, 'wm.nii.gz'))

In [34]:
lambda1 = (trace / 3.) * (1 + 2 * fa / (3 - 2 * fa ** 2) ** (1 / 2.))
lambda2 = (trace / 3.) * (1 - fa / (3 - 2 * fa ** 2) ** (1 / 2.))
evals = np.array([lambda1, lambda2, lambda2])

In [35]:
#show initial RF

res_obj = (evals, S0)

interactive = True

ren = window.Renderer()
evals = res_obj[0]
evecs = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]).T

sphere = get_sphere('symmetric724')
from dipy.sims.voxel import single_tensor_odf
response_odf = single_tensor_odf(sphere.vertices, evals, evecs)
# transform our data from 1D to 4D
response_odf = response_odf[None, None, None, :]
response_actor = actor.odf_slicer(response_odf, sphere=sphere, colormap='plasma')
ren.add(response_actor)

window.record(ren, out_path='fat_response.png', size=(200, 200))
if interactive:
    window.show(ren)

In [10]:
# Initialize the model
# response=res_obj
csd_model = ConstrainedSphericalDeconvModel(gtab, response)
sphere = get_sphere('symmetric724')
csd_peaks = peaks_from_model(model=csd_model,
                             data=data_small,
                             sphere=sphere,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             mask=mask_small, return_odf=True, return_sh=True)
ren = window.Renderer()
interactive = True
fodf_spheres = actor.odf_slicer(csd_peaks.odf, sphere=sphere, scale=0.6, colormap='plasma')
fodf_spheres.display_extent(0, 40, 5, 5, 0, 30)
ren.set_camera(position=(0, 10, 0))
ren.add(fodf_spheres)
if interactive:
    window.show(ren)

NameError: name 'response' is not defined

In [36]:
n = np.arange(0, sh_order + 1, 2)
where_dwi = lazy_index(~gtab.b0s_mask)
response_p = np.ones(len(n))
res_obj=(evals, S0)
r_sh_all=None
shape = data.shape[:-1]
data_rc = data[wm_mask]
print'The number of initial voxels is'+str(data_rc.shape)

indices = zip(*np.where(wm_mask==True))
    
for _ in range(iter):
    
    sf_voxels = np.zeros(shape)
    for idx in indices:
            sf_voxels[idx]=1
    nib.save(nib.Nifti1Image(sf_voxels.astype('float32'), img.affine, img.header), 
             os.path.join(wd, str(_)+'sf_voxels.nii.gz')) 

    
    print 'Iteration: '+str((_))
    r_sh_all = np.zeros(len(n))
    csd_model = ConstrainedSphericalDeconvModel(gtab, res_obj, sh_order=sh_order)
    csd_peaks = peaks_from_model(model=csd_model, data=data_rc, sphere=sphere,
                                 relative_peak_threshold=peak_thr, 
                                 min_separation_angle=min_sep_angle, parallel=False)

    #single peaks mask
    dirs = csd_peaks.peak_dirs
    vals = csd_peaks.peak_values
    print'shape of vals is: '+str(vals.shape)
    single_peak_mask = (vals[:, 1] / vals[:, 0]) < peak_thr
    print 'single mask shape is'+str(single_peak_mask.shape)
    data_rc = data_rc[single_peak_mask]
    print'data shape is'+str(data_rc.shape)
    dirs = dirs[single_peak_mask]
    
    new_indices=[]
    for i in range(len(indices)):
        if single_peak_mask[i]==True:
            new_indices.append(indices[i])
    indices = new_indices
    
    #reorient diffusion signal so thet the estimated fibers lies along z
    for num_vox in range(data_rc.shape[0]):
        rotmat = vec2vec_rotmat(dirs[num_vox, 0], np.array([0, 0, 1]))
        rot_gradients = np.dot(rotmat, gtab.gradients.T).T
        x, y, z = rot_gradients[where_dwi].T
        r, theta, phi = cart2sphere(x, y, z)
        B_dwi = real_sph_harm(0, n, theta[:, None], phi[:, None])
        r_sh_all += np.linalg.lstsq(B_dwi, data_rc[num_vox, where_dwi], rcond=-1)[0]
    
    print('updating response')
    response = r_sh_all / data_rc.shape[0]
    res_obj = AxSymShResponse(data_rc[:, gtab.b0s_mask].mean(), response)
 
    print('visualize response')
    interactive = False
    sphere = get_sphere('symmetric724')
    response_signal = res_obj.on_sphere(sphere)
    # transform our data from 1D to 4D
    response_signal = response_signal[None, None, None, :]
    response_actor = actor.odf_slicer(response_signal, sphere=sphere, colormap='plasma')
    ren = window.Renderer()
    ren.add(response_actor)
    print('Saving illustration as csd_recursive_response.png')
    window.record(ren, out_path=str(_)+'rf.png', size=(200, 200))
    if interactive:
        window.show(ren)

    print('checking convergence')
    change = abs((response_p - response) / response_p)
    print'change in response is'+str(change)
    if all(change < convergence):
        break
        
    response_p = response 

The number of initial voxels is(74934, 272)
Iteration: 0
shape of vals is: (74934, 5)
single mask shape is(74934,)
data shape is(68432, 272)
updating response
visualize response
Saving illustration as csd_recursive_response.png
checking convergence
change in response is[ 193.11474977   60.28688142   23.02107749   10.19402128    2.25861309]
Iteration: 1
shape of vals is: (68432, 5)
single mask shape is(68432,)
data shape is(17307, 272)
updating response
visualize response
Saving illustration as csd_recursive_response.png
checking convergence
change in response is[ 0.0229868   0.36261189  0.60472651  0.81075337  1.07272633]
Iteration: 2
shape of vals is: (17307, 5)
single mask shape is(17307,)
data shape is(7298, 272)
updating response
visualize response
Saving illustration as csd_recursive_response.png
checking convergence
change in response is[ 0.02497905  0.1221003   0.17031242  0.21687586  0.24842614]
Iteration: 3
shape of vals is: (7298, 5)
single mask shape is(7298,)
data shape is(

In [62]:
response

array([ 80.52063379, -32.7245135 ,  23.19480477, -14.85681182,   8.54864693])

In [63]:
res_obj = AxSymShResponse(data_rc[:, gtab.b0s_mask].mean(), response)

#check the shape of the signal of the response function, which should be like a pancake:
sphere = get_sphere('symmetric724')
response_signal = res_obj.on_sphere(sphere)
# transform our data from 1D to 4D
response_signal = response_signal[None, None, None, :]
response_actor = actor.odf_slicer(
    response_signal, sphere=sphere, colormap='plasma')
interactive = True
ren = window.Renderer()

ren.add(response_actor)
if interactive:
    window.show(ren)

In [12]:
#Initialize the model
csd_model = ConstrainedSphericalDeconvModel(gtab, res_obj)
sphere = get_sphere('symmetric724')
csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             mask=mask, return_odf=True, return_sh=True)

# save odf
nib.save(nib.Nifti1Image(csd_peaks.shm_coeff.astype(np.float32),
                                  img.affine, img.header), 'odf_csd_fa02_pr01_fa08.nii.gz')

In [13]:
#Enables/disables interactive visualization
ren = window.Renderer()
interactive = True
fodf_spheres = actor.odf_slicer(csd_peaks.odf, sphere=sphere, scale=0.6, norm=False, colormap='plasma')
fodf_spheres.display_extent(0, 140, 81, 81, 0, 96)
ren.set_camera(position=(0, 10, 0))
ren.add(fodf_spheres)

# print('Saving illustration as csd_odfs.png')
# window.record(ren, out_path='csd_fods.png', size=(600, 600))
if interactive:
    window.show(ren)

In [30]:
#tractography parameters
step_size = 0.75
max_angle= 45
pmf_thr=0.1
seeds_count=5
affine=np.eye(4)
sphere = get_sphere('symmetric724')
from dipy.reconst.shm import sh_to_sf
fimg = ("odf_csd_fa04_pr03_bodycc.nii.gz")
img = nib.load(fimg)
odfs = img.get_data()
seed_mask=("wm_mask_bin.nii.gz")
imgm = nib.load(seed_mask)
seed_mask = imgm.get_data()

img_pve_gm = nib.load('t12fa_pve_1_res.nii.gz')
img_pve_csf = nib.load('t12fa_pve_0_res.nii.gz')
img_pve_wm = nib.load('t12fa_pve_2_res.nii.gz')
seeds = random_seeds_from_mask(seed_mask, seeds_count=5)
from dipy.tracking.local import ParticleFilteringTracking
from dipy.tracking.local import CmcTissueClassifier
voxel_size = np.average(img_pve_wm.get_header()['pixdim'][1:4])
prob_dg = ProbabilisticDirectionGetter.from_shcoeff(odfs, max_angle=45, sphere=sphere)
cmc_classifier = CmcTissueClassifier.from_pve(img_pve_wm.get_data(),
                                              img_pve_gm.get_data(),
                                              img_pve_csf.get_data(),
                                              step_size=step_size,
                                              average_voxel_size=voxel_size)

all_streamlines_act_classifier = LocalTracking(prob_dg,
                                               cmc_classifier,
                                               seeds, 
                                               affine=np.eye(4), step_size=0.75,
                                               return_all=True)

streamlines = Streamlines(all_streamlines_act_classifier)
save_trk("odf_csd_fa04_pr03_bodycc.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
